In [143]:
import pandas as pd
import numpy as np
from scipy.stats import entropy
from pandas import Timedelta
from unidecode import unidecode
from scipy.stats import entropy
import gc
import time

# # Encoder para tratamento de variáveis categóricas
import category_encoders as ce

In [144]:
gc.enable()
def calcular_entropia(lista):
    valores, contagem = np.unique(lista, return_counts=True)
    return entropy(contagem, base=2)

In [145]:
dia = 'arquivos_base\df_lances_dia_3'
diretorio_destino = 'arquivos_treino_teste\Formato_por_dia\df_lances_dia3.csv'

In [146]:
df = pd.read_csv(dia)
df = df.fillna('Desconhecido')
# df = df[df.groupby('id_participante')['id_lance'].transform(lambda x: x.count()) >= 2]
# df = df.reset_index(drop=True)

In [147]:
# Calculos de entropia

# Convert relevant columns to strings
df['ip'] = df['ip'].astype(str)
df['pais'] = df['pais'].astype(str)
df['dispositivo'] = df['dispositivo'].astype(str)
df['url'] = df['url'].astype(str)

# Calculos de entropia
# Entropia IP
df_entropia = df.groupby('id_participante')['ip'].apply(calcular_entropia).reset_index()
df_entropia.columns = ['id_participante', 'entropia_ip']
df = df.merge(df_entropia, on='id_participante', how='left')

# Entropia País
df_entropia = df.groupby('id_participante')['pais'].apply(calcular_entropia).reset_index()
df_entropia.columns = ['id_participante', 'entropia_pais']
df = df.merge(df_entropia, on='id_participante', how='left')

# Entropia Dispositivo
df_entropia = df.groupby('id_participante')['dispositivo'].apply(calcular_entropia).reset_index()
df_entropia.columns = ['id_participante', 'entropia_dispositivo']
df = df.merge(df_entropia, on='id_participante', how='left')

# Entropia URL
df_entropia = df.groupby('id_participante')['url'].apply(calcular_entropia).reset_index()
df_entropia.columns = ['id_participante', 'entropia_url']
df = df.merge(df_entropia, on='id_participante', how='left')

# Entropia Pais - IP
df['pais_ip'] = df['pais'] + '_' + df['ip']
df_entropia = df.groupby('id_participante')['pais_ip'].apply(calcular_entropia).reset_index()
df_entropia.columns = ['id_participante', 'entropia_pais_ip']
df = df.merge(df_entropia, on='id_participante', how='left')

# Entropia Dispositivo - IP
df['dispositivo_ip'] = df['dispositivo'] + '_' + df['ip']
df_entropia = df.groupby('id_participante')['dispositivo_ip'].apply(calcular_entropia).reset_index()
df_entropia.columns = ['id_participante', 'entropia_dispositivo_ip']
df = df.merge(df_entropia, on='id_participante', how='left')

# Entropia Dispositivo - URL
df['dispositivo_url'] = df['dispositivo'] + '_' + df['url']
df_entropia = df.groupby('id_participante')['dispositivo_url'].apply(calcular_entropia).reset_index()
df_entropia.columns = ['id_participante', 'entropia_dispositivo_url']
df = df.merge(df_entropia, on='id_participante', how='left')

# Entropia Dispositivo - Pais
df['dispositivo_pais'] = df['dispositivo'] + '_' + df['pais']
df_entropia = df.groupby('id_participante')['dispositivo_pais'].apply(calcular_entropia).reset_index()
df_entropia.columns = ['id_participante', 'entropia_dispositivo_pais']
df = df.merge(df_entropia, on='id_participante', how='left')

# Entropia URL - IP
df['url_pais'] = df['url'] + '_' + df['pais']
df_entropia = df.groupby('id_participante')['url_pais'].apply(calcular_entropia).reset_index()
df_entropia.columns = ['id_participante', 'entropia_url_pais']
df = df.merge(df_entropia, on='id_participante', how='left')

gc.collect()


0

In [148]:
df_entropia = df.drop(columns = ['dispositivo', 'pais', 'ip', 'url', 'pais_ip', 'dispositivo_ip', 'dispositivo_url', 'dispositivo_pais', 'url_pais', 'id_lance', 'leilao', 'mercadoria', 'tempo'])
df_entropia = df_entropia.drop_duplicates(subset='id_participante', keep='first')
df_entropia = df_entropia.reset_index(drop=True)

In [149]:
##### Tempo #######


df = pd.read_csv(dia)
df = df.fillna('Desconhecido')
# df = df[df.groupby('id_participante')['id_lance'].transform(lambda x: x.count()) >= 2]
# df = df.reset_index(drop=True)

In [150]:
um_dia = 63663157894736
uma_hora = um_dia / 24
um_minuto = uma_hora // 60
um_segundo = um_minuto / 60

In [151]:
# Definindo a janela de tempo (em minutos)
window_size = int(30 * um_minuto)  # Exemplo: janela de 30 minutos


# Função para calcular a densidade de lances
def calcular_densidade_lances(dados, window_size):
    dados = dados.sort_values(by='tempo')
    dados['densidade'] = dados['tempo'].rolling(window=window_size, min_periods=1).count()
    return dados

df = calcular_densidade_lances(df, window_size)

# Identificar o pico de lances por leilão
df['pico'] = df.groupby('leilao')['densidade'].transform('max') == df['densidade']

# Otimização para verificar se um lance está no pico (retornando 0 ou 1)
def verificar_lance_no_pico(df, window_size):
    pico_lances = df[df['pico']].groupby('leilao')['tempo'].first().reset_index()
    pico_lances['intervalo_inicio'] = pico_lances['tempo'] - window_size // 2
    pico_lances['intervalo_fim'] = pico_lances['tempo'] + window_size // 2
    
    df = df.merge(pico_lances[['leilao', 'intervalo_inicio', 'intervalo_fim']], on='leilao', how='left')
    df['lance_no_pico'] = (df['tempo'] >= df['intervalo_inicio']) & (df['tempo'] <= df['intervalo_fim'])
    df['lance_no_pico'] = df['lance_no_pico'].astype(int)
    df.drop(columns=['intervalo_inicio', 'intervalo_fim'], inplace=True)
    
    return df

# Aplicar a função a todos os lances
df = verificar_lance_no_pico(df, window_size)
df = df.drop(columns=['densidade', 'pico'], axis = 1)

gc.collect()

0

In [152]:
# Calcular a quantidade de lances dentro e fora do pico por participante
df['qtde_lances_pico'] = df.groupby('id_participante')['lance_no_pico'].transform('sum')
df['qtde_lances_fora_pico'] = df.groupby('id_participante')['lance_no_pico'].transform(lambda x: (x == 0).sum())

# Calcular a porcentagem de lances no pico e fora dele
df['porcentagem_lances_pico'] = df['qtde_lances_pico'] / (df['qtde_lances_pico'] + df['qtde_lances_fora_pico'])
df['porcentagem_lances_fora_pico'] = 1 - df['porcentagem_lances_pico']

# Remover a coluna lance_no_pico
df.drop(columns=['lance_no_pico'], inplace=True)


gc.collect()

0

In [153]:
# fazer tratamento de tempo. Quartis que o usuário faz lances e o restante

# Calcular o tempo do leilao - Tempo Início e Tempo fim
tempos_leiloes = df.groupby(['leilao'])['tempo'].agg(['min', 'max']).reset_index()
tempos_leiloes.columns = ['leilao', 'inicio_leilao', 'fim_leilao']

# Passo 1: Juntar df e tempos_leiloes pelo código do leilão
df = df.merge(tempos_leiloes, on='leilao', how='left')

# Passo 2: Calcular a posição do tempo do lance em relação ao intervalo de tempo do leilão
df['posicao_relativa'] = (df['tempo'] - df['inicio_leilao']) / (df['fim_leilao'] - df['inicio_leilao'])

# Passo 3: Classificar a posição relativa em quartis
df['quartil_tempo'] = pd.qcut(df['posicao_relativa'], 4, labels=[1, 2, 3, 4])
contagem_quartis = df.groupby(['id_participante', 'quartil_tempo']).size().unstack(fill_value=0)

# Passo 4: Contar a distribuição dos quartis para cada participante
contagem_quartis = df.groupby(['id_participante', 'quartil_tempo']).size().unstack(fill_value=0)


# Renomear as colunas para facilitar a junção com df
contagem_quartis.columns = ['qtde_lances_quartil_1', 'qtde_lances_quartil_2', 'qtde_lances_quartil_3', 'qtde_lances_quartil_4']

contagem_quartis['soma_lances'] = contagem_quartis.sum(axis=1)
contagem_quartis['porcentagem_lances_quartil_1'] = contagem_quartis['qtde_lances_quartil_1'] / contagem_quartis['soma_lances']
contagem_quartis['porcentagem_lances_quartil_2'] = contagem_quartis['qtde_lances_quartil_2'] / contagem_quartis['soma_lances']
contagem_quartis['porcentagem_lances_quartil_3'] = contagem_quartis['qtde_lances_quartil_3'] / contagem_quartis['soma_lances']
contagem_quartis['porcentagem_lances_quartil_4'] = contagem_quartis['qtde_lances_quartil_4'] / contagem_quartis['soma_lances']


contagem_quartis['std_lances_por_quartil'] = contagem_quartis.iloc[:, :-1].std(axis=1)


# Resetar o índice para transformar contagem_quartis em um DataFrame regular
contagem_quartis = contagem_quartis.reset_index()
contagem_quartis.drop(columns=['soma_lances'], inplace=True)

# Adicionar as colunas de contagem de quartis ao df através de um merge
df = df.merge(contagem_quartis, on='id_participante', how='left')


## Calculando métricas de diferença de tempo

# Calcular a diferença de tempo entre lances

df = df.sort_values(['id_participante', 'tempo', 'id_lance', 'leilao'])
dif_tempo = df.groupby('id_participante')[['tempo']].diff()
df['diferenca_tempo'] = dif_tempo

# Métricas para diferença de tempo de lances do participante // depois fazer dif de tempo por leilao

df['media_diftempo_porusuario'] = df.groupby(['id_participante'])['diferenca_tempo'].transform('mean')
df['mediana_difs_tempo_participante'] = df.groupby('id_participante')['diferenca_tempo'].transform('median')

# Calcular desvio padrão da diferença de tempo entre lances

# Passo 1: Calcular a diferença quadrada entre lances por dispositivo e a média de lances de por dispositivo
df['diferenca_quadrada'] = (df['diferenca_tempo'] - df['media_diftempo_porusuario'])**2
# Passo 2: Agrupar novamente pelo id_participante e calcular a média dos quadrados das diferenças
desvios_quadraticos_medios = df.groupby('id_participante')['diferenca_quadrada'].mean()
# Passo 3: Tirar a raiz quadrada dessa média
std_difstempo_porlance = np.sqrt(desvios_quadraticos_medios)
std_difstempo_dict = std_difstempo_porlance.to_dict()

# Passo 4: Mapeando o desvio padrão correspondente para cada id_participante em df
df['std_difstempo_porlance'] = df['id_participante'].map(std_difstempo_dict)

# Tem lances ao mesmo tempo

# Passo 1: Contar quantos lances por participante ocorrem em cada tempo
df['qtde_lances_simultaneos'] = df.groupby(['id_participante', 'tempo'])['tempo'].transform('count')

# Passo 2: Criar a coluna 'tem_lance_simultaneo'
df['tem_lance_simultaneo'] = df['qtde_lances_simultaneos'].apply(lambda x: 1 if x > 1 else 0)

gc.collect()

C:\Users\andre\AppData\Local\Temp\ipykernel_12796\2867994732.py:15: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  contagem_quartis = df.groupby(['id_participante', 'quartil_tempo']).size().unstack(fill_value=0)
C:\Users\andre\AppData\Local\Temp\ipykernel_12796\2867994732.py:18: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  contagem_quartis = df.groupby(['id_participante', 'quartil_tempo']).size().unstack(fill_value=0)


0

In [154]:
pd.set_option('display.max_rows', None)
df.isna().sum()

id_lance                              0
id_participante                       0
leilao                                0
mercadoria                            0
dispositivo                           0
tempo                                 0
pais                                  0
ip                                    0
url                                   0
qtde_lances_pico                      0
qtde_lances_fora_pico                 0
porcentagem_lances_pico               0
porcentagem_lances_fora_pico          0
inicio_leilao                         0
fim_leilao                            0
posicao_relativa                    577
quartil_tempo                       577
qtde_lances_quartil_1                 0
qtde_lances_quartil_2                 0
qtde_lances_quartil_3                 0
qtde_lances_quartil_4                 0
porcentagem_lances_quartil_1          1
porcentagem_lances_quartil_2          1
porcentagem_lances_quartil_3          1
porcentagem_lances_quartil_4          1


In [155]:
df_tempo = df.drop_duplicates(subset='id_participante', keep='first')

colunas_para_dropar = ['id_lance', 'leilao', 'mercadoria', 'dispositivo', 'tempo', 'pais', 'ip', 'url', 'inicio_leilao', 'fim_leilao', 'posicao_relativa', 'quartil_tempo',
 'qtde_lances_quartil_1', 'qtde_lances_quartil_2', 'qtde_lances_quartil_3', 'qtde_lances_quartil_4', 'diferenca_tempo', 'diferenca_quadrada']

df_tempo = df_tempo.drop(columns=colunas_para_dropar, axis = 1)
df_tempo = df_tempo.reset_index(drop=True)

pd.set_option('display.max_rows', None)
df_tempo.isna().sum()


id_participante                      0
qtde_lances_pico                     0
qtde_lances_fora_pico                0
porcentagem_lances_pico              0
porcentagem_lances_fora_pico         0
porcentagem_lances_quartil_1         1
porcentagem_lances_quartil_2         1
porcentagem_lances_quartil_3         1
porcentagem_lances_quartil_4         1
std_lances_por_quartil               0
media_diftempo_porusuario          471
mediana_difs_tempo_participante    471
std_difstempo_porlance             471
qtde_lances_simultaneos              0
tem_lance_simultaneo                 0
dtype: int64

In [156]:
##### Feature Engineering Geral #######

df = pd.read_csv(dia)
df = df.fillna('Desconhecido')
# df = df[df.groupby('id_participante')['id_lance'].transform(lambda x: x.count()) >= 2]
# df = df.reset_index(drop=True)

gc.collect()


0

In [157]:
# Lances 
print('Inicio Lances')
inicio = time.time()

df['total_lances_participante'] = df.groupby(['id_participante'])['id_lance'].transform('count')
df['leiloes_participados'] = df.groupby(['id_participante'])['leilao'].transform('nunique')
df['total_lances_nesseleilao'] = df.groupby(['id_participante', 'leilao'])['id_lance'].transform('count')
df['media_lances_porleilao'] = df.groupby(['id_participante'])['total_lances_nesseleilao'].transform('mean')
df['max_lances_leilao'] = df.groupby(['id_participante'])['total_lances_nesseleilao'].transform('max')

## Calculando mediana de lances por leilão
# Agrupando e transformando as listas de lances por leilão em cada grupo
valores_mediana_porleilao = df.groupby(['id_participante'])['total_lances_nesseleilao'].agg(lambda x: np.median(x.tolist())).reset_index()
# Renomeando a coluna da mediana
valores_mediana_porleilao.columns = ['id_participante', 'mediana_lances_porleilao']
# Mesclando com o dataframe original
df = pd.merge(df, valores_mediana_porleilao, on='id_participante', how='left')


# Calcular desvio padrão de lances por leilão
# Passo 1: Calcular a diferença quadrada entre lances do leilao e a média 
df['diferenca_quadrada'] = (df['total_lances_nesseleilao'] - df['media_lances_porleilao'])**2
# Passo 2: Agrupar novamente pelo id_participante e calcular a média dos quadrados das diferenças
desvios_quadraticos_medios = df.groupby('id_participante')['diferenca_quadrada'].mean()
# Passo 3: Tirar a raiz quadrada dessa média
std_lances_porleilao = np.sqrt(desvios_quadraticos_medios)
# Passo 4: Mapeando o desvio padrão correspondente para cada id_participante em df
df['std_lances_porleilao'] = df['id_participante'].map(std_lances_porleilao)
df = df.drop(columns = 'diferenca_quadrada', axis = 1)


fim = time.time()
print(f'Fim Lances - {fim - inicio} segundos')


print('Inicio Mercadorias')
inicio = time.time()


Inicio Lances
Fim Lances - 6.485780954360962 segundos
Inicio Mercadorias


In [158]:
# Mercadoria
df['qtde_mercadorias_participante'] = df.groupby(['id_participante'])['mercadoria'].transform('nunique')

# Loop para quantidade de artigos esportivos:
lista_mercadorias = ['artigos esportivos', 'joias', 'móveis',
       'equipamentos de escritório', 'bens domésticos', 'livros e música',
       'computadores', 'peças de automóveis', 'vestuário']

for m in lista_mercadorias:
    df_artigos_esportivos = df[df['mercadoria'] == m]
    contagem_lances = df_artigos_esportivos.groupby('id_participante')['id_lance'].count()
    nome_coluna = 'qtde_lance_' + m
    df[nome_coluna] = df['id_participante'].map(contagem_lances).fillna(0).astype(int)

# 2. OneHotEncoder na variável 'mercadoria'
encoder = ce.OneHotEncoder(cols=['mercadoria'], use_cat_names=True)
mercadoria_encoded = encoder.fit_transform(df)

mercadoria_encoded_selected = mercadoria_encoded[['id_participante'] + [col for col in mercadoria_encoded.columns if col.startswith('mercadoria_')]]

# 3. Agregar os resultados para cada 'id_participante'
result_df = mercadoria_encoded_selected.groupby('id_participante').max().reset_index()

# 4. Fazer o merge dos resultados agregados com o DataFrame original df
df = df.merge(result_df, on='id_participante', how='left')

fim = time.time()
print(f'Fim Mercadorias - {fim - inicio} segundos')

gc.collect()

Fim Mercadorias - 20.95551061630249 segundos


0

In [159]:
# Dispositivo

print('Inicio Dispositivo')
inicio = time.time()

df['total_dispositivos_participante'] = df.groupby(['id_participante'])['dispositivo'].transform('nunique')

# Lances por dispositivo

df['dispositivos_duplicados'] = df.groupby('id_participante')['dispositivo'].transform(lambda x: x.duplicated(keep=False).sum())
df['total_lances_pordispositivo'] = df.groupby(['id_participante', 'dispositivo'])['id_lance'].transform('nunique')
df['max_lances_pordispositivo'] = df.groupby(['id_participante'])['total_lances_pordispositivo'].transform('max')
df['media_lances_pordispositivo'] = df.groupby(['id_participante'])['total_lances_pordispositivo'].transform('mean')

## Calculando mediana de lances por dispositivo
# Agrupando e transformando as listas de lances por por dispositivo em cada grupo
valores_mediana_lances_pordispositivo = df.groupby(['id_participante'])['total_lances_pordispositivo'].agg(lambda x: np.median(x.tolist())).reset_index()
# Renomeando a coluna da mediana
valores_mediana_lances_pordispositivo.columns = ['id_participante', 'mediana_lances_pordispositivo']

# Mesclando com o dataframe original
df = pd.merge(df, valores_mediana_lances_pordispositivo, on='id_participante', how='left')

# Calcular desvio padrão de lances por dispositivo

# Passo 1: Calcular a diferença quadrada entre lances por dispositivo e a média de lances de por dispositivo
df['diferenca_quadrada'] = (df['total_lances_pordispositivo'] - df['media_lances_pordispositivo'])**2
# Passo 2: Agrupar novamente pelo id_participante e calcular a média dos quadrados das diferenças
desvios_quadraticos_medios = df.groupby('id_participante')['diferenca_quadrada'].mean()
# Passo 3: Tirar a raiz quadrada dessa média
std_lances_porleilao = np.sqrt(desvios_quadraticos_medios)
# Passo 4: Mapeando o desvio padrão correspondente para cada id_participante em df
df['std_lances_pordispositivo'] = df['id_participante'].map(std_lances_porleilao)
df = df.drop(columns = 'diferenca_quadrada', axis = 1)


# Dispositivos por leilão

df['total_dispositivos_porleilao'] = df.groupby(['id_participante', 'leilao'])['dispositivo'].transform('nunique')
df['max_dispositivos_porleilao'] = df.groupby(['id_participante'])['total_dispositivos_porleilao'].transform('max')
df['media_dispositivos_porleilao'] = df.groupby(['id_participante'])['total_lances_pordispositivo'].transform('mean')

## Calculando mediana dispositivos por leilão
# Agrupando e transformando as listas de dispositivo por leilão em cada grupo
valores_mediana_dispositivo_porleilao = df.groupby(['id_participante'])['total_dispositivos_porleilao'].agg(lambda x: np.median(x.tolist())).reset_index()
# Renomeando a coluna da mediana
valores_mediana_dispositivo_porleilao.columns = ['id_participante', 'mediana_dispositivo_porleilao']
# Mesclando com o dataframe original
df = pd.merge(df, valores_mediana_dispositivo_porleilao, on='id_participante', how='left')

# Calcular desvio padrão de dispositivos por leilão

# Passo 1: Calcular a diferença quadrada entre lances por dispositivo e a média de lances de por dispositivo
df['diferenca_quadrada'] = (df['total_dispositivos_porleilao'] - df['media_dispositivos_porleilao'])**2
# Passo 2: Agrupar novamente pelo id_participante e calcular a média dos quadrados das diferenças
desvios_quadraticos_medios = df.groupby('id_participante')['diferenca_quadrada'].mean()
# Passo 3: Tirar a raiz quadrada dessa média
std_lances_porleilao = np.sqrt(desvios_quadraticos_medios)
# Passo 4: Mapeando o desvio padrão correspondente para cada id_participante em df
df['std_dispositivos_porleilao'] = df['id_participante'].map(std_lances_porleilao)
df = df.drop(columns = 'diferenca_quadrada', axis = 1)


# Dispositivos por IP

df['total_dispositivos_porip'] = df.groupby(['id_participante', 'ip'])['dispositivo'].transform('nunique')
df['max_dispositivos_porip'] = df.groupby(['id_participante'])['total_dispositivos_porip'].transform('max')
df['media_dispositivos_porip'] = df.groupby(['id_participante'])['total_dispositivos_porip'].transform('mean')

## Calculando mediana dispositivos por leilão
# Agrupando e transformando as listas de dispositivo por leilão em cada grupo
valores_mediana_dispositivo_porip = df.groupby(['id_participante'])['total_dispositivos_porip'].agg(lambda x: np.median(x.tolist())).reset_index()
# Renomeando a coluna da mediana
valores_mediana_dispositivo_porip.columns = ['id_participante', 'mediana_dispositivo_porip']
# Mesclando com o dataframe original
df = pd.merge(df, valores_mediana_dispositivo_porip, on='id_participante', how='left')

# Calcular desvio padrão de dispositivos por ip

# Passo 1: Calcular a diferença quadrada entre lances por dispositivo e a média de lances de por dispositivo
df['diferenca_quadrada'] = (df['total_dispositivos_porip'] - df['media_dispositivos_porip'])**2
# Passo 2: Agrupar novamente pelo id_participante e calcular a média dos quadrados das diferenças
desvios_quadraticos_medios = df.groupby('id_participante')['diferenca_quadrada'].mean()
# Passo 3: Tirar a raiz quadrada dessa média
std_dispositivos_porip = np.sqrt(desvios_quadraticos_medios)
# Passo 4: Mapeando o desvio padrão correspondente para cada id_participante em df
df['std_dispositivos_porip'] = df['id_participante'].map(std_dispositivos_porip)
df = df.drop(columns = 'diferenca_quadrada', axis = 1)


# Dispositivos por URL

df['total_dispositivos_porurl'] = df.groupby(['id_participante', 'url'])['dispositivo'].transform('nunique')
df['max_dispositivos_porurl'] = df.groupby(['id_participante'])['total_dispositivos_porurl'].transform('max')
df['media_dispositivos_porurl'] = df.groupby(['id_participante'])['total_dispositivos_porurl'].transform('mean')

## Calculando mediana Dispositivos por URL
# Agrupando e transformando as listas de dispositivo por leilão em cada grupo
valores_mediana_dispositivo_porurl = df.groupby(['id_participante'])['total_dispositivos_porurl'].agg(lambda x: np.median(x.tolist())).reset_index()
# Renomeando a coluna da mediana
valores_mediana_dispositivo_porurl.columns = ['id_participante', 'mediana_dispositivo_porurl']
# Mesclando com o dataframe original
df = pd.merge(df, valores_mediana_dispositivo_porurl, on='id_participante', how='left')

# Calcular desvio padrão de Dispositivos por URL

# Passo 1: Calcular a diferença quadrada entre lances por dispositivo e a média de lances de por dispositivo
df['diferenca_quadrada'] = (df['total_dispositivos_porurl'] - df['media_dispositivos_porurl'])**2
# Passo 2: Agrupar novamente pelo id_participante e calcular a média dos quadrados das diferenças
desvios_quadraticos_medios = df.groupby('id_participante')['diferenca_quadrada'].mean()
# Passo 3: Tirar a raiz quadrada dessa média
std_dispositivos_porurl = np.sqrt(desvios_quadraticos_medios)
# Passo 4: Mapeando o desvio padrão correspondente para cada id_participante em df
df['std_dispositivos_porurl'] = df['id_participante'].map(std_dispositivos_porurl)
df = df.drop(columns = 'diferenca_quadrada', axis = 1)


fim = time.time()
print(f'Fim dispositivos - {fim - inicio} segundos')


Inicio Dispositivo
Fim dispositivos - 37.538708448410034 segundos


In [160]:
# IPs

print('Inicio IPs')
inicio = time.time()

df['total_ips_participante'] = df.groupby(['id_participante'])['ip'].transform('nunique')

# IPs por leilão

df['ips_duplicados'] = df.groupby('id_participante')['ip'].transform(lambda x: x.duplicated(keep=False).sum())
df['ips_por_leilao'] = df.groupby(['id_participante', 'leilao'])['ip'].transform('nunique')
df['max_ips_porleilao'] = df.groupby(['id_participante'])['ips_por_leilao'].transform('max')
df['media_ips_porleilao'] = df.groupby(['id_participante'])['ips_por_leilao'].transform('mean')

## Calculando mediana de IPs por leilao
# Agrupando e transformando as listas de lances por por dispositivo em cada grupo
valores_mediana_ips_porleilao = df.groupby(['id_participante'])['ips_por_leilao'].agg(lambda x: np.median(x.tolist())).reset_index()
# Renomeando a coluna da mediana
valores_mediana_ips_porleilao.columns = ['id_participante', 'mediana_ips_porleilao']
# Mesclando com o dataframe original
df = pd.merge(df, valores_mediana_ips_porleilao, on='id_participante', how='left')

# Calcular desvio padrão de IPs por leilão

# Passo 1: Calcular a diferença quadrada entre lances por dispositivo e a média de lances de por dispositivo
df['diferenca_quadrada'] = (df['ips_por_leilao'] - df['media_ips_porleilao'])**2
# Passo 2: Agrupar novamente pelo id_participante e calcular a média dos quadrados das diferenças
desvios_quadraticos_medios = df.groupby('id_participante')['diferenca_quadrada'].mean()
# Passo 3: Tirar a raiz quadrada dessa média
std_ips_porleilao = np.sqrt(desvios_quadraticos_medios)
# Passo 4: Mapeando o desvio padrão correspondente para cada id_participante em df
df['std_ips_porleilao'] = df['id_participante'].map(std_ips_porleilao)
df = df.drop(columns = 'diferenca_quadrada', axis = 1)


# Lances por IP

df['lances_por_ip'] = df.groupby(['id_participante', 'ip'])['id_lance'].transform('nunique')
df['max_lances_porip'] = df.groupby(['id_participante'])['lances_por_ip'].transform('max')
df['media_lances_porip'] = df.groupby(['id_participante'])['lances_por_ip'].transform('mean')

## Calculando mediana de lances por país
# Agrupando e transformando as listas de lances por por dispositivo em cada grupo
valores_mediana_lances_porip = df.groupby(['id_participante'])['lances_por_ip'].agg(lambda x: np.median(x.tolist())).reset_index()
# Renomeando a coluna da mediana
valores_mediana_lances_porip.columns = ['id_participante', 'mediana_lances_porip']
# Mesclando com o dataframe original
df = pd.merge(df, valores_mediana_lances_porip, on='id_participante', how='left')

# Calcular desvio padrão de lances por país

# Passo 1: Calcular a diferença quadrada entre lances por dispositivo e a média de lances de por dispositivo
df['diferenca_quadrada'] = (df['lances_por_ip'] - df['media_lances_porip'])**2
# Passo 2: Agrupar novamente pelo id_participante e calcular a média dos quadrados das diferenças
desvios_quadraticos_medios = df.groupby('id_participante')['diferenca_quadrada'].mean()
# Passo 3: Tirar a raiz quadrada dessa média
std_lances_porip = np.sqrt(desvios_quadraticos_medios)
# Passo 4: Mapeando o desvio padrão correspondente para cada id_participante em df
df['std_lances_porip'] = df['id_participante'].map(std_lances_porip)
df = df.drop(columns = 'diferenca_quadrada', axis = 1)


# IPs por dispositivo

df['ips_por_dispositivo'] = df.groupby(['id_participante', 'dispositivo'])['ip'].transform('nunique')
df['max_ips_pordispositivo'] = df.groupby(['id_participante'])['ips_por_dispositivo'].transform('max')
df['media_ips_pordispositivo'] = df.groupby(['id_participante'])['ips_por_dispositivo'].transform('mean')

## Calculando mediana de IPs por leilao
# Agrupando e transformando as listas de lances por por dispositivo em cada grupo
valores_mediana_ips_pordispositivo = df.groupby(['id_participante'])['ips_por_dispositivo'].agg(lambda x: np.median(x.tolist())).reset_index()
# Renomeando a coluna da mediana
valores_mediana_ips_pordispositivo.columns = ['id_participante', 'mediana_ips_por_dispositivo']
# Mesclando com o dataframe original
df = pd.merge(df, valores_mediana_ips_pordispositivo, on='id_participante', how='left')

# Calcular desvio padrão de IPs por dispositivo

# Passo 1: Calcular a diferença quadrada entre lances por dispositivo e a média de lances de por dispositivo
df['diferenca_quadrada'] = (df['ips_por_dispositivo'] - df['media_ips_pordispositivo'])**2
# Passo 2: Agrupar novamente pelo id_participante e calcular a média dos quadrados das diferenças
desvios_quadraticos_medios = df.groupby('id_participante')['diferenca_quadrada'].mean()
# Passo 3: Tirar a raiz quadrada dessa média
std_ips_pordispositivo = np.sqrt(desvios_quadraticos_medios)
# Passo 4: Mapeando o desvio padrão correspondente para cada id_participante em df
df['std_ips_pordispositivo'] = df['id_participante'].map(std_ips_pordispositivo)
df = df.drop(columns = 'diferenca_quadrada', axis = 1)


# IPs por país

df['ips_por_pais'] = df.groupby(['id_participante', 'pais'])['ip'].transform('nunique')
df['max_ips_porpais'] = df.groupby(['id_participante'])['ips_por_pais'].transform('max')
df['media_ips_porpais'] = df.groupby(['id_participante'])['ips_por_pais'].transform('mean')

## Calculando mediana de IPs por país
# Agrupando e transformando as listas de lances por por dispositivo em cada grupo
valores_mediana_ips_porpais = df.groupby(['id_participante'])['ips_por_pais'].agg(lambda x: np.median(x.tolist())).reset_index()
# Renomeando a coluna da mediana
valores_mediana_ips_porpais.columns = ['id_participante', 'mediana_ips_por_pais']
# Mesclando com o dataframe original
df = pd.merge(df, valores_mediana_ips_porpais, on='id_participante', how='left')

# Calcular desvio padrão de IPs por país

# Passo 1: Calcular a diferença quadrada entre lances por dispositivo e a média de lances de por dispositivo
df['diferenca_quadrada'] = (df['ips_por_pais'] - df['media_ips_porpais'])**2
# Passo 2: Agrupar novamente pelo id_participante e calcular a média dos quadrados das diferenças
desvios_quadraticos_medios = df.groupby('id_participante')['diferenca_quadrada'].mean()
# Passo 3: Tirar a raiz quadrada dessa média
std_ips_porpais = np.sqrt(desvios_quadraticos_medios)
# Passo 4: Mapeando o desvio padrão correspondente para cada id_participante em df
df['std_ips_porpais'] = df['id_participante'].map(std_ips_porpais)
df = df.drop(columns = 'diferenca_quadrada', axis = 1)


# IPs por URL

df['ips_por_url'] = df.groupby(['id_participante', 'url'])['ip'].transform('nunique')
df['max_ips_porurl'] = df.groupby(['id_participante'])['ips_por_url'].transform('max')
df['media_ips_porurl'] = df.groupby(['id_participante'])['ips_por_url'].transform('mean')

## Calculando mediana de IPs por URL
# Agrupando e transformando as listas de lances por por dispositivo em cada grupo
valores_mediana_ips_porurl = df.groupby(['id_participante'])['ips_por_url'].agg(lambda x: np.median(x.tolist())).reset_index()
# Renomeando a coluna da mediana
valores_mediana_ips_porurl.columns = ['id_participante', 'mediana_ips_por_url']
# Mesclando com o dataframe original
df = pd.merge(df, valores_mediana_ips_porurl, on='id_participante', how='left')

# Calcular desvio padrão de IPs por URL

# Passo 1: Calcular a diferença quadrada entre lances por dispositivo e a média de lances de por dispositivo
df['diferenca_quadrada'] = (df['ips_por_url'] - df['media_ips_porurl'])**2
# Passo 2: Agrupar novamente pelo id_participante e calcular a média dos quadrados das diferenças
desvios_quadraticos_medios = df.groupby('id_participante')['diferenca_quadrada'].mean()
# Passo 3: Tirar a raiz quadrada dessa média
std_ips_porurl = np.sqrt(desvios_quadraticos_medios)
# Passo 4: Mapeando o desvio padrão correspondente para cada id_participante em df
df['std_ips_porurl'] = df['id_participante'].map(std_ips_porurl)
df = df.drop(columns = 'diferenca_quadrada', axis = 1)

fim = time.time()
print(f'Fim IPs - {fim - inicio} segundos')

gc.collect()

Inicio IPs
Fim IPs - 53.38318610191345 segundos


0

In [161]:
# País

print('Inicio País')
inicio = time.time()

df['total_paises_participante'] = df.groupby(['id_participante'])['pais'].transform('nunique')

# País por leilão

df['paises_duplicados'] = df.groupby('id_participante')['pais'].transform(lambda x: x.duplicated(keep=False).sum())
df['paises_por_leilao'] = df.groupby(['id_participante', 'leilao'])['pais'].transform('nunique')
df['max_pais_porleilao'] = df.groupby(['id_participante'])['paises_por_leilao'].transform('max')
df['media_pais_porleilao'] = df.groupby(['id_participante'])['paises_por_leilao'].transform('mean')

## Calculando mediana de países por leilão
# Agrupando e transformando as listas de lances por por dispositivo em cada grupo
valores_mediana_paises_porleilao = df.groupby(['id_participante'])['paises_por_leilao'].agg(lambda x: np.median(x.tolist())).reset_index()
# Renomeando a coluna da mediana
valores_mediana_paises_porleilao.columns = ['id_participante', 'mediana_paises_por_leilao']
# Mesclando com o dataframe original
df = pd.merge(df, valores_mediana_paises_porleilao, on='id_participante', how='left')

# Calcular desvio padrão de países por leilão

# Passo 1: Calcular a diferença quadrada entre lances por dispositivo e a média de lances de por dispositivo
df['diferenca_quadrada'] = (df['paises_por_leilao'] - df['media_pais_porleilao'])**2
# Passo 2: Agrupar novamente pelo id_participante e calcular a média dos quadrados das diferenças
desvios_quadraticos_medios = df.groupby('id_participante')['diferenca_quadrada'].mean()
# Passo 3: Tirar a raiz quadrada dessa média
std_paises_porleilao = np.sqrt(desvios_quadraticos_medios)
# Passo 4: Mapeando o desvio padrão correspondente para cada id_participante em df
df['std_paises_porleilao'] = df['id_participante'].map(std_paises_porleilao)
df = df.drop(columns = 'diferenca_quadrada', axis = 1)


# Lances por país

df['lances_por_pais'] = df.groupby(['id_participante', 'pais'])['id_lance'].transform('nunique')
df['max_lances_porpais'] = df.groupby(['id_participante'])['lances_por_pais'].transform('max')
df['media_lances_porpais'] = df.groupby(['id_participante'])['lances_por_pais'].transform('mean')

## Calculando mediana de lances por país
# Agrupando e transformando as listas de lances por por dispositivo em cada grupo
valores_mediana_lances_porpais = df.groupby(['id_participante'])['lances_por_pais'].agg(lambda x: np.median(x.tolist())).reset_index()
# Renomeando a coluna da mediana
valores_mediana_lances_porpais.columns = ['id_participante', 'mediana_lances_porpais']
# Mesclando com o dataframe original
df = pd.merge(df, valores_mediana_lances_porpais, on='id_participante', how='left')

# Calcular desvio padrão de lances por país

# Passo 1: Calcular a diferença quadrada entre lances por dispositivo e a média de lances de por dispositivo
df['diferenca_quadrada'] = (df['lances_por_pais'] - df['media_lances_porpais'])**2
# Passo 2: Agrupar novamente pelo id_participante e calcular a média dos quadrados das diferenças
desvios_quadraticos_medios = df.groupby('id_participante')['diferenca_quadrada'].mean()
# Passo 3: Tirar a raiz quadrada dessa média
std_lances_porpais = np.sqrt(desvios_quadraticos_medios)
# Passo 4: Mapeando o desvio padrão correspondente para cada id_participante em df
df['std_lances_porpais'] = df['id_participante'].map(std_lances_porpais)
df = df.drop(columns = 'diferenca_quadrada', axis = 1)


# País por dispositivo

df['paises_por_dispositivo'] = df.groupby(['id_participante', 'dispositivo'])['pais'].transform('nunique')
df['max_pais_pordispositivo'] = df.groupby(['id_participante'])['paises_por_dispositivo'].transform('max')
df['media_pais_pordispositivo'] = df.groupby(['id_participante'])['paises_por_dispositivo'].transform('mean')

## Calculando mediana de países por leilão
# Agrupando e transformando as listas de lances por por dispositivo em cada grupo
valores_mediana_paises_pordispositivo = df.groupby(['id_participante'])['paises_por_dispositivo'].agg(lambda x: np.median(x.tolist())).reset_index()
# Renomeando a coluna da mediana
valores_mediana_paises_pordispositivo.columns = ['id_participante', 'mediana_paises_por_dispositivo']
# Mesclando com o dataframe original
df = pd.merge(df, valores_mediana_paises_pordispositivo, on='id_participante', how='left')

# Calcular desvio padrão de países por leilão

# Passo 1: Calcular a diferença quadrada entre lances por dispositivo e a média de lances de por dispositivo
df['diferenca_quadrada'] = (df['paises_por_dispositivo'] - df['media_pais_pordispositivo'])**2
# Passo 2: Agrupar novamente pelo id_participante e calcular a média dos quadrados das diferenças
desvios_quadraticos_medios = df.groupby('id_participante')['diferenca_quadrada'].mean()
# Passo 3: Tirar a raiz quadrada dessa média
std_paises_pordispositivo = np.sqrt(desvios_quadraticos_medios)
# Passo 4: Mapeando o desvio padrão correspondente para cada id_participante em df
df['std_paises_pordispositivo'] = df['id_participante'].map(std_paises_pordispositivo)
df = df.drop(columns = 'diferenca_quadrada', axis = 1)


# País por url

df['paises_por_url'] = df.groupby(['id_participante', 'url'])['pais'].transform('nunique')
df['max_pais_porurl'] = df.groupby(['id_participante'])['paises_por_url'].transform('max')
df['media_pais_porurl'] = df.groupby(['id_participante'])['paises_por_url'].transform('mean')

## Calculando mediana de países por url
# Agrupando e transformando as listas de lances por por dispositivo em cada grupo
valores_mediana_paises_porurl = df.groupby(['id_participante'])['paises_por_url'].agg(lambda x: np.median(x.tolist())).reset_index()
# Renomeando a coluna da mediana
valores_mediana_paises_porurl.columns = ['id_participante', 'mediana_paises_por_url']
# Mesclando com o dataframe original
df = pd.merge(df, valores_mediana_paises_porurl, on='id_participante', how='left')

# Calcular desvio padrão de países por url

# Passo 1: Calcular a diferença quadrada entre lances por dispositivo e a média de lances de por dispositivo
df['diferenca_quadrada'] = (df['paises_por_url'] - df['media_pais_porurl'])**2
# Passo 2: Agrupar novamente pelo id_participante e calcular a média dos quadrados das diferenças
desvios_quadraticos_medios = df.groupby('id_participante')['diferenca_quadrada'].mean()
# Passo 3: Tirar a raiz quadrada dessa média
std_paises_porurl = np.sqrt(desvios_quadraticos_medios)
# Passo 4: Mapeando o desvio padrão correspondente para cada id_participante em df
df['std_paises_porurl'] = df['id_participante'].map(std_paises_porurl)
df = df.drop(columns = 'diferenca_quadrada', axis = 1)

# País por ip

df['paises_por_ip'] = df.groupby(['id_participante', 'ip'])['pais'].transform('nunique')
df['max_pais_ip'] = df.groupby(['id_participante'])['paises_por_ip'].transform('max')
df['media_pais_porip'] = df.groupby(['id_participante'])['paises_por_ip'].transform('mean')

## Calculando mediana de países por ip
# Agrupando e transformando as listas de lances por por dispositivo em cada grupo
valores_mediana_paises_porip = df.groupby(['id_participante'])['paises_por_ip'].agg(lambda x: np.median(x.tolist())).reset_index()
# Renomeando a coluna da mediana
valores_mediana_paises_porip.columns = ['id_participante', 'mediana_paises_por_ip']
# Mesclando com o dataframe original
df = pd.merge(df, valores_mediana_paises_porip, on='id_participante', how='left')

# Calcular desvio padrão de países por ip

# Passo 1: Calcular a diferença quadrada entre lances por dispositivo e a média de lances de por dispositivo
df['diferenca_quadrada'] = (df['paises_por_ip'] - df['media_pais_porip'])**2
# Passo 2: Agrupar novamente pelo id_participante e calcular a média dos quadrados das diferenças
desvios_quadraticos_medios = df.groupby('id_participante')['diferenca_quadrada'].mean()
# Passo 3: Tirar a raiz quadrada dessa média
std_paises_porip = np.sqrt(desvios_quadraticos_medios)
# Passo 4: Mapeando o desvio padrão correspondente para cada id_participante em df
df['std_paises_porip'] = df['id_participante'].map(std_paises_porip)
df = df.drop(columns = 'diferenca_quadrada', axis = 1)


fim = time.time()
print(f'Fim Pais - {fim - inicio} segundos')


Inicio País
Fim Pais - 56.58388829231262 segundos


In [162]:
# URL

print('Inicio URLs')
inicio = time.time()

df['total_urls_participante'] = df.groupby(['id_participante'])['url'].transform('nunique')

# URLs por leilão

df['urls_duplicados'] = df.groupby('id_participante')['url'].transform(lambda x: x.duplicated(keep=False).sum())
df['urls_por_leilao'] = df.groupby(['id_participante', 'leilao'])['url'].transform('nunique')
df['max_urls_porleilao'] = df.groupby(['id_participante'])['urls_por_leilao'].transform('max')
df['media_urls_porleilao'] = df.groupby(['id_participante'])['urls_por_leilao'].transform('mean')

## Calculando mediana de URLs por leilao
# Agrupando e transformando as listas de lances por por dispositivo em cada grupo
valores_mediana_urls_porleilao = df.groupby(['id_participante'])['urls_por_leilao'].agg(lambda x: np.median(x.tolist())).reset_index()
# Renomeando a coluna da mediana
valores_mediana_urls_porleilao.columns = ['id_participante', 'mediana_urls_por_leilao']
# Mesclando com o dataframe original
df = pd.merge(df, valores_mediana_urls_porleilao, on='id_participante', how='left')

# Calcular desvio padrão de URLs por leilão

# Passo 1: Calcular a diferença quadrada entre lances por dispositivo e a média de lances de por dispositivo
df['diferenca_quadrada'] = (df['urls_por_leilao'] - df['media_urls_porleilao'])**2
# Passo 2: Agrupar novamente pelo id_participante e calcular a média dos quadrados das diferenças
desvios_quadraticos_medios = df.groupby('id_participante')['diferenca_quadrada'].mean()
# Passo 3: Tirar a raiz quadrada dessa média
std_urls_porleilao = np.sqrt(desvios_quadraticos_medios)
# Passo 4: Mapeando o desvio padrão correspondente para cada id_participante em df
df['std_urls_porleilao'] = df['id_participante'].map(std_urls_porleilao)
df = df.drop(columns = 'diferenca_quadrada', axis = 1)


# Lances por URL

df['lances_por_url'] = df.groupby(['id_participante', 'url'])['id_lance'].transform('nunique')
df['max_lances_porurl'] = df.groupby(['id_participante'])['lances_por_url'].transform('max')
df['media_lances_porurl'] = df.groupby(['id_participante'])['lances_por_url'].transform('mean')

## Calculando mediana de lances por URL
# Agrupando e transformando as listas de lances por por dispositivo em cada grupo
valores_mediana_lances_porurl = df.groupby(['id_participante'])['lances_por_url'].agg(lambda x: np.median(x.tolist())).reset_index()
# Renomeando a coluna da mediana
valores_mediana_lances_porurl.columns = ['id_participante', 'mediana_lances_porurl']
# Mesclando com o dataframe original
df = pd.merge(df, valores_mediana_lances_porurl, on='id_participante', how='left')

# Calcular desvio padrão de lances por URL

# Passo 1: Calcular a diferença quadrada entre lances por dispositivo e a média de lances de por dispositivo
df['diferenca_quadrada'] = (df['lances_por_url'] - df['media_lances_porurl'])**2
# Passo 2: Agrupar novamente pelo id_participante e calcular a média dos quadrados das diferenças
desvios_quadraticos_medios = df.groupby('id_participante')['diferenca_quadrada'].mean()
# Passo 3: Tirar a raiz quadrada dessa média
std_lances_porurl = np.sqrt(desvios_quadraticos_medios)
# Passo 4: Mapeando o desvio padrão correspondente para cada id_participante em df
df['std_lances_porurl'] = df['id_participante'].map(std_lances_porurl)
df = df.drop(columns = 'diferenca_quadrada', axis = 1)


# URLs por dispositivo
df['urls_por_dispositivo'] = df.groupby(['id_participante', 'dispositivo'])['url'].transform('nunique')
df['max_urls_pordispositivo'] = df.groupby(['id_participante'])['urls_por_dispositivo'].transform('max')
df['media_urls_pordispositivo'] = df.groupby(['id_participante'])['urls_por_dispositivo'].transform('mean')

## Calculando mediana de URLs por dispositivo
# Agrupando e transformando as listas de lances por por dispositivo em cada grupo
valores_mediana_urls_pordispositivo = df.groupby(['id_participante'])['urls_por_dispositivo'].agg(lambda x: np.median(x.tolist())).reset_index()
# Renomeando a coluna da mediana
valores_mediana_urls_pordispositivo.columns = ['id_participante', 'mediana_urls_por_dispositivo']
# Mesclando com o dataframe original
df = pd.merge(df, valores_mediana_urls_pordispositivo, on='id_participante', how='left')

# Calcular desvio padrão de URLs por Dispositivo

# Passo 1: Calcular a diferença quadrada entre lances por dispositivo e a média de lances de por dispositivo
df['diferenca_quadrada'] = (df['urls_por_dispositivo'] - df['media_urls_pordispositivo'])**2
# Passo 2: Agrupar novamente pelo id_participante e calcular a média dos quadrados das diferenças
desvios_quadraticos_medios = df.groupby('id_participante')['diferenca_quadrada'].mean()
# Passo 3: Tirar a raiz quadrada dessa média
std_urls_pordispositivo = np.sqrt(desvios_quadraticos_medios)
# Passo 4: Mapeando o desvio padrão correspondente para cada id_participante em df
df['std_urls_pordispositivo'] = df['id_participante'].map(std_urls_pordispositivo)
df = df.drop(columns = 'diferenca_quadrada', axis = 1)


# URLs por pais
df['urls_por_pais'] = df.groupby(['id_participante', 'pais'])['url'].transform('nunique')
df['max_urls_porpais'] = df.groupby(['id_participante'])['urls_por_pais'].transform('max')
df['media_urls_porpais'] = df.groupby(['id_participante'])['urls_por_pais'].transform('mean')

## Calculando mediana de URLs por pais
# Agrupando e transformando as listas de lances por por dispositivo em cada grupo
valores_mediana_urls_porpais = df.groupby(['id_participante'])['urls_por_pais'].agg(lambda x: np.median(x.tolist())).reset_index()
# Renomeando a coluna da mediana
valores_mediana_urls_porpais.columns = ['id_participante', 'mediana_urls_por_pais']
# Mesclando com o dataframe original
df = pd.merge(df, valores_mediana_urls_porpais, on='id_participante', how='left')

# Calcular desvio padrão de URLs por pais

# Passo 1: Calcular a diferença quadrada entre lances por dispositivo e a média de lances de por dispositivo
df['diferenca_quadrada'] = (df['urls_por_pais'] - df['media_urls_porpais'])**2
# Passo 2: Agrupar novamente pelo id_participante e calcular a média dos quadrados das diferenças
desvios_quadraticos_medios = df.groupby('id_participante')['diferenca_quadrada'].mean()
# Passo 3: Tirar a raiz quadrada dessa média
std_urls_porpais = np.sqrt(desvios_quadraticos_medios)
# Passo 4: Mapeando o desvio padrão correspondente para cada id_participante em df
df['std_urls_porpais'] = df['id_participante'].map(std_urls_porpais)
df = df.drop(columns = 'diferenca_quadrada', axis = 1)


# URLs por ip
df['urls_por_ip'] = df.groupby(['id_participante', 'ip'])['url'].transform('nunique')
df['max_urls_porip'] = df.groupby(['id_participante'])['urls_por_ip'].transform('max')
df['media_urls_porip'] = df.groupby(['id_participante'])['urls_por_ip'].transform('mean')

## Calculando mediana de URLs por ip
# Agrupando e transformando as listas de lances por por dispositivo em cada grupo
valores_mediana_urls_porip = df.groupby(['id_participante'])['urls_por_ip'].agg(lambda x: np.median(x.tolist())).reset_index()
# Renomeando a coluna da mediana
valores_mediana_urls_porip.columns = ['id_participante', 'mediana_urls_por_ip']
# Mesclando com o dataframe original
df = pd.merge(df, valores_mediana_urls_porip, on='id_participante', how='left')

# Calcular desvio padrão de URLs por ip

# Passo 1: Calcular a diferença quadrada entre lances por dispositivo e a média de lances de por dispositivo
df['diferenca_quadrada'] = (df['urls_por_ip'] - df['media_urls_porip'])**2
# Passo 2: Agrupar novamente pelo id_participante e calcular a média dos quadrados das diferenças
desvios_quadraticos_medios = df.groupby('id_participante')['diferenca_quadrada'].mean()
# Passo 3: Tirar a raiz quadrada dessa média
std_urls_porip = np.sqrt(desvios_quadraticos_medios)
# Passo 4: Mapeando o desvio padrão correspondente para cada id_participante em df
df['std_urls_porip'] = df['id_participante'].map(std_urls_porip)
df = df.drop(columns = 'diferenca_quadrada', axis = 1)

fim = time.time()
print(f'Fim URLs - {fim - inicio} segundos')

gc.collect()


Inicio URLs
Fim URLs - 65.96945333480835 segundos


0

In [163]:
colunas_para_dropar = ['total_lances_nesseleilao', 'mercadoria_artigos esportivos', 'mercadoria_joias', 'mercadoria_móveis', 'mercadoria_equipamentos de escritório',
 'mercadoria_bens domésticos', 'mercadoria_livros e música', 'mercadoria_computadores', 'mercadoria_peças de automóveis', 'mercadoria_vestuário',
 'total_lances_pordispositivo', 'total_dispositivos_porleilao', 'total_dispositivos_porip', 'total_dispositivos_porurl', 'ips_por_leilao',
 'lances_por_ip', 'ips_por_dispositivo', 'ips_por_pais', 'ips_por_url', 'paises_por_leilao', 'lances_por_pais', 'paises_por_dispositivo',
 'paises_por_url', 'urls_por_leilao', 'lances_por_url', 'urls_por_dispositivo', 'urls_por_pais', 'urls_por_ip']

df = df.drop(columns=colunas_para_dropar, axis = 1)
df = df.drop_duplicates(subset='id_participante', keep='first')
df = df.reset_index(drop=True)


In [164]:
# Ratios e novas estatísticas (Lances vs todos)


# Lances por leilão

print('Inicio Lances vs todos')
inicio = time.time()

# Ratio media / mediana Lances por leilão
df['ratio_media_mediana_lances_porleilao'] = df['media_lances_porleilao'] / df['mediana_lances_porleilao']

# Ratio entre desvio padrão e média
df['ratio_std_media_lances_porleilao'] = df['std_lances_porleilao'] / df['media_lances_porleilao']

# Combinação média e mediana focando em detecção de outliers
df['comb_media_mediana_lances_porleilao'] = (0.5 * df['media_lances_porleilao']) + (0.5 * df['mediana_lances_porleilao'])

# Lances por dipositivo

# Ratio Max e média de lances por dispositivo
df['ratio_max_media_lances_pordispositivo'] = df['max_lances_pordispositivo'] / df['media_lances_pordispositivo']

# Diferença entre max e mediana de lances por dispositivo
df['diff_max_mediana_lances_pordispositivo'] = df['max_lances_pordispositivo'] - df['mediana_lances_pordispositivo']
df['var_percentual_media_mediana_lances_porleilao'] = (df['media_lances_porleilao'] - df['mediana_lances_pordispositivo']) / df['mediana_lances_pordispositivo']

# Produto média e std de lances por dispositivo
df['interacao_media_std_lances_pordispositivo'] = df['media_lances_pordispositivo'] * df['std_lances_pordispositivo']


# Lances por IP
# Ratio Max e média de lances por ip
df['ratio_max_media_lances_porip'] = df['max_lances_porip'] / df['media_lances_porip']

# Diferença entre max e mediana de lances por ip

df['diff_max_mediana_lances_porip'] = df['max_lances_porip'] - df['mediana_lances_porip']
df['var_percentual_media_mediana_lances_porip'] = (df['media_lances_porip'] - df['mediana_lances_porip']) / df['mediana_lances_porip']

# Produto média e std de lances por ip
df['interacao_media_std_lances_porip'] = df['media_lances_porip'] * df['std_lances_porip']

# Ratio Max e média de ips por leilao
df['ratio_max_media_lances_porip'] = df['max_lances_porip'] / df['media_lances_porip']


# Lances por URL
# Ratio Max e média de Lances por URL
df['ratio_max_media_lances_porurl'] = df['max_lances_porurl'] / df['media_lances_porurl']

# Diferença entre max e mediana de Lances por URL

df['diff_max_mediana_lances_porurl'] = df['max_lances_porurl'] - df['mediana_lances_porurl']
df['var_percentual_media_mediana_lances_porurl'] = (df['media_lances_porurl'] - df['mediana_lances_porurl']) / df['mediana_lances_porurl']

# Produto média e std de Lances por URL
df['interacao_media_std_lances_porurl'] = df['media_lances_porurl'] * df['std_lances_porurl']

# Ratio Max e média de Lances por URL
df['ratio_max_media_lances_porurl'] = df['max_lances_porurl'] / df['media_lances_porurl']


# Lances por país
# Ratio Max e média de Lances por país
df['ratio_max_media_lances_porpais'] = df['max_lances_porpais'] / df['media_lances_porpais']

# Diferença entre max e mediana de Lances por país

df['diff_max_mediana_lances_porpais'] = df['max_lances_porpais'] - df['mediana_lances_porpais']
df['var_percentual_media_mediana_lances_porpais'] = (df['media_lances_porpais'] - df['mediana_lances_porpais']) / df['mediana_lances_porpais']

# Produto média e std de Lances por país
df['interacao_media_std_lances_porurl'] = df['media_lances_porpais'] * df['std_lances_porpais']

# Ratio Max e média de Lances por país
df['ratio_max_media_lances_porpais'] = df['max_lances_porpais'] / df['media_lances_porpais']

fim = time.time()
print(f'Fim Lances vs Todos - {fim - inicio} segundos')

Inicio Lances vs todos
Fim Lances vs Todos - 0.015793561935424805 segundos


In [165]:
# Ratios e novas estatísticas (dispositivo vs todos)

print('Inicio dispositivos vs todos')
inicio = time.time()

# Dispositivos por leilão

# Ratio media / mediana Lances por leilão
df['ratio_media_mediana_dispositivos_porleilao'] = df['media_dispositivos_porleilao'] / df['mediana_dispositivo_porleilao']

# Ratio entre desvio padrão e média
df['ratio_std_media_dispositivos_porleilao'] = df['std_dispositivos_porleilao'] / df['media_dispositivos_porleilao']

# Combinação média e mediana focando em detecção de outliers
df['comb_media_mediana_dispositivos_porleilao'] = (0.5 * df['media_dispositivos_porleilao']) + (0.5 * df['mediana_dispositivo_porleilao'])


# Dispositivos por IP
# Ratio Max e média de Dispositivos por IP
df['ratio_max_media_dispositivos_porip'] = df['max_dispositivos_porip'] / df['media_dispositivos_porip']

# Diferença entre max e mediana de Dispositivos por IP

df['diff_max_mediana_dispositivos_porip'] = df['max_dispositivos_porip'] - df['mediana_dispositivo_porip']
df['var_percentual_media_mediana_dispositivos_porip'] = (df['media_dispositivos_porip'] - df['mediana_dispositivo_porip']) / df['mediana_dispositivo_porip']

# Produto média e std de Dispositivos por IP
df['interacao_media_std_dispositivos_porip'] = df['media_dispositivos_porip'] * df['std_dispositivos_porip']

# Ratio Max e média de Dispositivos por IP
df['ratio_max_media_dispositivos_porip'] = df['max_dispositivos_porip'] / df['media_dispositivos_porip']


# Dispositivos por URL
# Ratio Max e média de Dispositivos por URL
df['ratio_max_media_dispositivos_porurl'] = df['max_dispositivos_porurl'] / df['mediana_dispositivo_porurl']

# Diferença entre max e mediana de Dispositivos por URL

df['diff_max_mediana_dispositivos_porurl'] = df['max_dispositivos_porurl'] - df['mediana_dispositivo_porurl']
df['var_percentual_media_mediana_dispositivos_porurl'] = (df['media_dispositivos_porurl'] - df['mediana_dispositivo_porurl']) / df['mediana_dispositivo_porurl']

# Produto média e std de Dispositivos por URL
df['interacao_media_std_dispositivos_porurl'] = df['media_dispositivos_porurl'] * df['std_dispositivos_porurl']

# Ratio Max e média de Dispositivos por URL
df['ratio_max_media_dispositivos_porurl'] = df['max_dispositivos_porurl'] / df['media_dispositivos_porurl']

fim = time.time()
print(f'Fim Dispositivos vs todos - {fim - inicio} segundos')

gc.collect()

Inicio dispositivos vs todos
Fim Dispositivos vs todos - 0.010057687759399414 segundos


0

In [166]:
# Ratios e novas estatísticas (pais vs todos)

print('Inicio País vs todos')
inicio = time.time()


# Ratio Max e média de Lances por país
df['ratio_max_media_lances_porpais'] = df['max_lances_porpais'] / df['media_lances_porpais']

# Diferença entre max e mediana de Lances por país

df['diff_max_mediana_lances_porpais'] = df['max_lances_porpais'] - df['mediana_lances_porpais']
df['var_percentual_media_mediana_lances_porpais'] = (df['media_lances_porpais'] - df['mediana_lances_porpais']) / df['mediana_lances_porpais']

# Produto média e std de Lances por país
df['interacao_media_std_lances_porurl'] = df['media_lances_porpais'] * df['std_lances_porpais']

# Ratio Max e média de Lances por país
df['ratio_max_media_lances_porpais'] = df['max_lances_porpais'] / df['media_lances_porpais']

# Pais por leilão

# Ratio media / mediana Pais por leilão
df['ratio_media_mediana_pais_porleilao'] = df['media_pais_porleilao'] / df['mediana_paises_por_leilao']

# Ratio entre desvio padrão e média
df['ratio_std_media_pais_porleilao'] = df['std_paises_porleilao'] / df['media_pais_porleilao']

# Combinação média e mediana focando em detecção de outliers
df['comb_media_mediana_pais_porleilao'] = (0.5 * df['media_pais_porleilao']) + (0.5 * df['mediana_paises_por_leilao'])


# Pais por dispositivo
# Ratio Max e média de pais por dispositivo
df['ratio_max_media_pais_pordispositivo'] = df['max_pais_pordispositivo'] / df['media_pais_pordispositivo']

# Diferença entre max e mediana de Pais por dispositivo

df['diff_max_mediana_pais_pordispositivo'] = df['max_pais_pordispositivo'] - df['mediana_paises_por_dispositivo']
df['var_percentual_media_mediana_pais_pordispositivo'] = (df['media_pais_pordispositivo'] - df['mediana_paises_por_dispositivo']) / df['mediana_paises_por_dispositivo']

# Produto média e std de Pais por dispositivo
df['interacao_media_pais_pordispositivo'] = df['media_pais_pordispositivo'] * df['std_paises_pordispositivo']

# Ratio Max e média de Pais por dispositivo
df['ratio_max_media_pais_pordispositivo'] = df['max_pais_pordispositivo'] / df['media_pais_pordispositivo']


# Pais por IP
# Ratio Max e média de Pais por IP
df['ratio_max_media_pais_porip'] = df['max_pais_ip'] / df['media_pais_porip']

# Diferença entre max e mediana de Pais por IP

df['diff_max_mediana_pais_porip'] = df['max_pais_ip'] - df['mediana_paises_por_ip']
df['var_percentual_media_mediana_pais_porip'] = (df['media_pais_porip'] - df['mediana_paises_por_ip']) / df['mediana_paises_por_ip']

# Produto média e std de Pais por IP
df['interacao_media_std_pais_porip'] = df['media_pais_porip'] * df['std_paises_porip']

# Ratio Max e média de Pais por IP
df['ratio_max_media_pais_porip'] = df['max_pais_ip'] / df['media_pais_porip']


# Pais por URL
# Ratio Max e média de pais por URL
df['ratio_max_media_pais_porurl'] = df['max_pais_porurl'] / df['media_pais_porurl']

# Diferença entre max e mediana de pais por URL

df['diff_max_mediana_pais_porurl'] = df['max_pais_porurl'] - df['mediana_paises_por_url']
df['var_percentual_media_mediana_pais_porurl'] = (df['media_pais_porurl'] - df['mediana_paises_por_url']) / df['mediana_paises_por_url']

# Produto média e std de pais por URL
df['interacao_media_std_pais_porurl'] = df['media_pais_porurl'] * df['std_paises_porurl']

# Ratio Max e média de Dispositivos por URL
df['ratio_max_media_pais_porurl'] = df['max_pais_porurl'] / df['media_pais_porurl']

fim = time.time()
print(f'Fim País vs todos - {fim - inicio} segundos')

Inicio País vs todos
Fim País vs todos - 0.016762495040893555 segundos


In [167]:
# Ratios e novas estatísticas (ips vs todos)

print('Inicio IPs vs todos')
inicio = time.time()

# IPs por leilão

# Ratio media / mediana Pais por leilão
df['ratio_media_mediana_ips_porleilao'] = df['media_ips_porleilao'] / df['mediana_ips_porleilao']

# Ratio entre desvio padrão e média
df['ratio_std_media_ips_porleilao'] = df['std_ips_porleilao'] / df['media_ips_porleilao']

# Combinação média e mediana focando em detecção de outliers
df['comb_media_mediana_ips_porleilao'] = (0.5 * df['media_ips_porleilao']) + (0.5 * df['mediana_ips_porleilao'])


# IPs por dispositivo
# Ratio Max e média de pais por dispositivo
df['ratio_max_media_ips_pordispositivo'] = df['max_ips_pordispositivo'] / df['media_ips_pordispositivo']

# Diferença entre max e mediana de Pais por dispositivo

df['diff_max_mediana_ips_pordispositivo'] = df['max_ips_pordispositivo'] - df['mediana_ips_por_dispositivo']
df['var_percentual_media_mediana_ips_pordispositivo'] = (df['media_ips_pordispositivo'] - df['mediana_ips_por_dispositivo']) / df['mediana_ips_por_dispositivo']

# Produto média e std de Pais por dispositivo
df['interacao_media_ips_pordispositivo'] = df['media_ips_pordispositivo'] * df['std_ips_pordispositivo']

# Ratio Max e média de Pais por dispositivo
df['ratio_max_media_ips_pordispositivo'] = df['max_ips_pordispositivo'] / df['media_ips_pordispositivo']


# IPs por país
# Ratio Max e média de IPs por país
df['ratio_max_media_ips_porpais'] = df['max_ips_porpais'] / df['media_ips_porpais']

# Diferença entre max e mediana de IPs por país

df['diff_max_mediana_ips_porpais'] = df['max_ips_porpais'] - df['mediana_ips_por_pais']
df['var_percentual_media_mediana_ips_porpais'] = (df['media_ips_porpais'] - df['mediana_ips_por_pais']) / df['mediana_ips_por_pais']

# Produto média e std de IPs por país
df['interacao_media_std_ips_porpais'] = df['media_ips_porpais'] * df['std_ips_porpais']

# Ratio Max e média de IPs por país
df['ratio_max_media_ips_porpais'] = df['max_ips_porpais'] / df['media_ips_porpais']


# IPs por URL
# Ratio Max e média de pais por URL
df['ratio_max_media_ips_porurl'] = df['max_ips_porurl'] / df['media_ips_porurl']

# Diferença entre max e mediana de IPs por URL

df['diff_max_mediana_ips_porurl'] = df['max_ips_porurl'] - df['mediana_ips_por_url']
df['var_percentual_media_mediana_ips_porurl'] = (df['media_ips_porurl'] - df['mediana_ips_por_url']) / df['mediana_ips_por_url']

# Produto média e std de IPs por URL
df['interacao_media_std_ips_porurl'] = df['media_ips_porurl'] * df['std_ips_porurl']

# Ratio Max e média de IPs por URL
df['ratio_max_media_ips_porurl'] = df['max_ips_porurl'] / df['media_ips_porurl']

fim = time.time()
print(f'Fim IPs vs todos - {fim - inicio} segundos')

Inicio IPs vs todos
Fim IPs vs todos - 0.014189958572387695 segundos


In [168]:
# Ratios e novas estatísticas (URLs vs todos)

print('Inicio URLs vs todos')
inicio = time.time()

# URLs por leilão

# Ratio media / mediana URL por leilão
df['ratio_media_mediana_urls_porleilao'] = df['media_urls_porleilao'] / df['mediana_urls_por_leilao']

# Ratio entre desvio padrão e média
df['ratio_std_media_urls_porleilao'] = df['std_urls_porleilao'] / df['media_urls_porleilao']

# Combinação média e mediana focando em detecção de outliers
df['comb_media_mediana_urls_porleilao'] = (0.5 * df['media_urls_porleilao']) + (0.5 * df['mediana_urls_por_leilao'])


# URLs por dispositivo
# Ratio Max e média de pais por dispositivo
df['ratio_max_media_urls_pordispositivo'] = df['max_urls_pordispositivo'] / df['media_urls_pordispositivo']

# Diferença entre max e mediana de URL por dispositivo

df['diff_max_mediana_urls_pordispositivo'] = df['max_urls_pordispositivo'] - df['mediana_urls_por_dispositivo']
df['var_percentual_media_mediana_urls_pordispositivo'] = (df['media_urls_pordispositivo'] - df['mediana_urls_por_dispositivo']) / df['mediana_urls_por_dispositivo']

# Produto média e std de URLs por dispositivo
df['interacao_media_urls_pordispositivo'] = df['media_urls_pordispositivo'] * df['std_urls_pordispositivo']

# Ratio Max e média de URLs por dispositivo
df['ratio_max_media_urls_pordispositivo'] = df['max_urls_pordispositivo'] / df['media_urls_pordispositivo']


# URLs por país
# Ratio Max e média de IPs por país
df['ratio_max_media_urls_porpais'] = df['max_urls_porpais'] / df['media_urls_porpais']

# Diferença entre max e mediana de IPs por país

df['diff_max_mediana_urls_porpais'] = df['max_urls_porpais'] - df['mediana_urls_por_pais']
df['var_percentual_media_mediana_urls_porpais'] = (df['media_urls_porpais'] - df['mediana_urls_por_pais']) / df['mediana_urls_por_pais']

# Produto média e std de IPs por país
df['interacao_media_std_urls_porpais'] = df['media_urls_porpais'] * df['std_urls_porpais']

# Ratio Max e média de IPs por país
df['ratio_max_media_urls_porpais'] = df['max_urls_porpais'] / df['media_urls_porpais']


# URLs por IP
# Ratio Max e média de URLs por IP
df['ratio_max_media_urls_porip'] = df['max_urls_porip'] / df['media_urls_porip']

# Diferença entre max e mediana de URLs por IP

df['diff_max_mediana_urls_porip'] = df['max_urls_porip'] - df['mediana_urls_por_ip']
df['var_percentual_media_mediana_urls_porip'] = (df['media_urls_porip'] - df['mediana_urls_por_ip']) / df['mediana_urls_por_ip']

# Produto média e std de URLs por IP
df['interacao_media_std_urls_porip'] = df['media_urls_porip'] * df['std_urls_porip']

# Ratio Max e média de URLs por IP
df['ratio_max_media_urls_porip'] = df['max_urls_porip'] / df['media_urls_porip']

fim = time.time()
print(f'Fim URLs vs todos - {fim - inicio} segundos')

gc.collect()


Inicio URLs vs todos
Fim URLs vs todos - 0.01493525505065918 segundos


0

In [169]:
colunas_para_dropar = ['id_lance', 'leilao', 'mercadoria', 'dispositivo', 'tempo', 'pais', 'ip', 'url']
df.drop(columns=colunas_para_dropar, axis=1, inplace=True)
df = df.drop_duplicates(subset='id_participante', keep='first')
df = df.reset_index(drop=True)

df_lances = df.merge(df_tempo, on='id_participante', how='left')
df_lances = df_lances.merge(df_entropia, on='id_participante', how='left')

df_lances.to_csv(diretorio_destino, index= False)



In [170]:
df_lances.columns.tolist()

['id_participante',
 'total_lances_participante',
 'leiloes_participados',
 'media_lances_porleilao',
 'max_lances_leilao',
 'mediana_lances_porleilao',
 'std_lances_porleilao',
 'qtde_mercadorias_participante',
 'qtde_lance_artigos esportivos',
 'qtde_lance_joias',
 'qtde_lance_móveis',
 'qtde_lance_equipamentos de escritório',
 'qtde_lance_bens domésticos',
 'qtde_lance_livros e música',
 'qtde_lance_computadores',
 'qtde_lance_peças de automóveis',
 'qtde_lance_vestuário',
 'total_dispositivos_participante',
 'dispositivos_duplicados',
 'max_lances_pordispositivo',
 'media_lances_pordispositivo',
 'mediana_lances_pordispositivo',
 'std_lances_pordispositivo',
 'max_dispositivos_porleilao',
 'media_dispositivos_porleilao',
 'mediana_dispositivo_porleilao',
 'std_dispositivos_porleilao',
 'max_dispositivos_porip',
 'media_dispositivos_porip',
 'mediana_dispositivo_porip',
 'std_dispositivos_porip',
 'max_dispositivos_porurl',
 'media_dispositivos_porurl',
 'mediana_dispositivo_porurl'